<a href="https://www.kaggle.com/code/mdrashidshahariar/candlesense?scriptVersionId=282684193" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
from kaggle_secrets import UserSecretsClient


try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


In [2]:
import os
import json
import time
import uuid
import warnings
import subprocess
import requests
import numpy as np
import pandas as pd
import yfinance as yf

In [3]:
from google.adk.agents import Agent, LlmAgent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner, Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import google_search, AgentTool, FunctionTool, ToolContext, load_memory, preload_memory
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from google.adk.models.google_llm import Gemini
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent, AGENT_CARD_WELL_KNOWN_PATH
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.apps.app import App, ResumabilityConfig

from google.genai.types import Content, Part 

import warnings
warnings.filterwarnings("ignore")
print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
# Cell 2: Candlestick Pattern Detection Functions
def identify_candlestick_patterns(df):
    """Detects common candlestick patterns without using external technical analysis libraries."""
    required_cols = ['Open', 'High', 'Low', 'Close']
    if not all(col in df.columns for col in required_cols):
        raise ValueError("DataFrame must contain Open, High, Low, Close columns")
    
    patterns = []
    
    for i in range(len(df)):
        row = df.iloc[i]
        o, h, l, c = row['Open'], row['High'], row['Low'], row['Close']
        body_size = abs(c - o)
        upper_shadow = h - max(o, c)
        lower_shadow = min(o, c) - l
        total_range = h - l
        
        if total_range == 0:
            patterns.append('No Pattern')
            continue
        
        pattern = 'No Pattern'
        
        # Doji pattern
        if body_size <= 0.1 * total_range and upper_shadow > 0.1 * total_range and lower_shadow > 0.1 * total_range:
            pattern = 'Doji'
        
        # Hammer and Hanging Man
        elif lower_shadow >= 2 * body_size and upper_shadow <= body_size * 0.3 and body_size <= 0.3 * total_range:
            pattern = 'Hammer' if c > o else 'Hanging Man'
        
        # Shooting Star and Inverted Hammer
        elif upper_shadow >= 2 * body_size and lower_shadow <= body_size * 0.3 and body_size <= 0.3 * total_range:
            pattern = 'Shooting Star' if c > o else 'Inverted Hammer'
        
        # Bullish Engulfing
        elif (i > 0 and df.iloc[i-1]['Close'] < df.iloc[i-1]['Open'] and 
              c > df.iloc[i-1]['Open'] and o < df.iloc[i-1]['Close'] and
              body_size > abs(df.iloc[i-1]['Close'] - df.iloc[i-1]['Open'])):
            pattern = 'Bullish Engulfing'
        
        # Bearish Engulfing
        elif (i > 0 and df.iloc[i-1]['Close'] > df.iloc[i-1]['Open'] and 
              c < df.iloc[i-1]['Open'] and o > df.iloc[i-1]['Close'] and
              body_size > abs(df.iloc[i-1]['Close'] - df.iloc[i-1]['Open'])):
            pattern = 'Bearish Engulfing'
        
        patterns.append(pattern)
    
    df = df.copy()
    df['Pattern'] = patterns
    return df

def get_pattern_signal(pattern):
    """Returns the typical trading signal for each pattern."""
    bullish_patterns = ['Hammer', 'Bullish Engulfing', 'Inverted Hammer']
    bearish_patterns = ['Hanging Man', 'Shooting Star', 'Bearish Engulfing']
    
    if pattern in bullish_patterns:
        return 'Bullish'
    elif pattern in bearish_patterns:
        return 'Bearish'
    elif pattern == 'Doji':
        return 'Neutral/Indecision'
    else:
        return 'No Clear Signal'

In [5]:
# Cell 3: Custom Function Tool for Data Fetching
def fetch_stock_data(symbol: str, period: str = "1mo") -> dict:
    """
    Fetches stock data and identifies candlestick patterns for a given symbol.
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'TSLA')
        period: Time period for data retrieval (default: '1mo')
    
    Returns:
        Dictionary containing latest price information and detected candlestick patterns
    """
    try:
        ticker = yf.Ticker(symbol)
        data = ticker.history(period=period)
        if data.empty:
            return {"error": f"No data available for symbol {symbol}"}
        
        data_with_patterns = identify_candlestick_patterns(data)
        
        recent_patterns = data_with_patterns.tail(5)[['Close', 'Pattern']].to_dict('records')
        
        summary = {
            'latest_price': float(data['Close'].iloc[-1]),
            'price_change': float(data['Close'].iloc[-1] - data['Close'].iloc[-2]),
            'recent_patterns': recent_patterns,
            'latest_pattern': data_with_patterns['Pattern'].iloc[-1],
            'pattern_signal': get_pattern_signal(data_with_patterns['Pattern'].iloc[-1]),
            'pattern_count': data_with_patterns['Pattern'].value_counts().to_dict()
        }
        
        return summary
    except Exception as e:
        return {"error": f"Error fetching data for {symbol}: {str(e)}"}

# Create the function tool
data_fetch_tool = FunctionTool(fetch_stock_data)
print("✅ Data fetching tool created successfully.")

✅ Data fetching tool created successfully.


In [6]:
# Cell 4: Agent Definitions (Corrected)

# Initialize the language model
llm = Gemini(model="gemini-1.5-pro")

# Define specialized agents
data_fetcher_agent = LlmAgent(
    name="DataFetcherAgent",
    model=llm,
    description="An agent specialized in retrieving stock market data and identifying candlestick patterns. It fetches recent price data and automatically detects patterns such as Doji, Hammer, Engulfing patterns, etc.",
    tools=[data_fetch_tool]
)

pattern_analyzer_agent = LlmAgent(
    name="PatternAnalyzerAgent",
    model=llm,
    description="An expert candlestick pattern analyst that evaluates the significance and reliability of identified patterns within the context of recent price action and market conditions.",
    tools=[data_fetch_tool]
)

trading_advisor_agent = LlmAgent(
    name="TradingAdvisorAgent",
    model=llm,
    description="A professional trading advisor that provides clear, actionable buy, hold, or sell recommendations based on candlestick pattern analysis, including risk assessment and confirmation requirements.",
    tools=[data_fetch_tool]
)

print("✅ All individual agents created successfully.")

# Note: Instead of creating a SequentialAgent with multiple agents, we will 
# coordinate the agents explicitly through sequential method calls.

✅ All individual agents created successfully.


In [7]:
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.runners import Runner

# Initialize services
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

# Create the Runner with app_name and agent (no App instance needed)
# Use data_fetcher_agent as the root agent for the runner
runner = Runner(
    app_name="TradingAnalysisApp",
    agent=data_fetcher_agent,
    session_service=session_service,
    memory_service=memory_service
)

print("✅ Session management and runner initialized with Runner.")

✅ Session management and runner initialized with Runner.


In [8]:
############# Extra cell – CORRECTED FOR KAGGLE (Legacy SDK)
# Correct import for google-generativeai (v0.x in Kaggle)
import google.generativeai as genai
Content = genai.protos.Content  # Legacy path for Content
Part = genai.protos.Part        # Legacy path for Part

# Robust helper function (fixes empty output)
def extract_text_from_events(events):
    text = ""
    for event in events:
        if hasattr(event, "content") and event.content:
            for part in event.content.parts:
                if hasattr(part, "text") and part.text:
                    text += part.text + " "
    return text.strip()

print("✅ FIXES APPLIED – Content, Part, and extract_text_from_events() are now defined! (Legacy SDK compatible)")


✅ FIXES APPLIED – Content, Part, and extract_text_from_events() are now defined! (Legacy SDK compatible)


In [9]:
# Cell 6: FINAL GUARANTEED WORKING — Short Prompt + No Truncation
import google.generativeai as genai
import json
import yfinance as yf

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    "gemini-2.5-flash",
    safety_settings=[
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    ]
)

def analyze_stock_sequential(symbol):
    try:
        # Always works — 3mo data
        df = yf.Ticker(symbol).history(period="3mo")
        if df.empty:
            return f"No data for {symbol}"
        df = identify_candlestick_patterns(df).tail(5)  # Only 5 candles — short & safe
        
        latest = df.iloc[-1]
        prev = df.iloc[-2]
        price = round(latest['Close'], 2)
        change = round(latest['Close'] - prev['Close'], 2)
        pattern = latest['Pattern']
        signal = get_pattern_signal(pattern)
        
        # Short, safe prompt — can't be blocked
        prompt = f"""Short chart summary for {symbol}:

Price: ${price} (change: {change:+})
Pattern: {pattern} (signal: {signal})

Summarize in 3 short sections:

1. DATA: Price and pattern
2. ANALYSIS: What it means (1 sentence)
3. NOTE: Key level to watch

Keep it under 100 words."""

        response = model.generate_content(
            prompt,
            generation_config={
                "temperature": 0.1,
                "max_output_tokens": 200  # Short to avoid truncation
            }
        )

        # Anti-truncation extraction
        text = ""
        if response and response.candidates:
            for part in response.candidates[0].content.parts:
                if hasattr(part, 'text') and part.text:
                    text += part.text + " "
        if not text.strip():
            text = f"1. DATA: Price ${price} ({change:+}) | Pattern {pattern}\n2. ANALYSIS: {pattern} indicates {signal.lower()} bias.\n3. NOTE: Watch support at ${round(price * 0.98, 2)}"

        return f"""Analysis for {symbol}
Price: ${price} ({change:+})

{text}"""

    except Exception as e:
        return f"Error: {str(e)}"

print("FINAL SHORT VERSION READY — Run Cell 7 for complete output!")

FINAL SHORT VERSION READY — Run Cell 7 for complete output!


In [10]:
# Cell 7: Final Live Demo
print("=== Candlestick Pattern Trading Analysis System ===\n")

for symbol in ["AAPL", "TSLA"]:
    print("\n" + "="*80)
    print(f"CANDLESTICK PATTERN ANALYSIS FOR {symbol}")
    print("="*80)
    
    result = analyze_stock_sequential(symbol)
    print(result)
    
    print("\n" + "-"*80 + "\n")

=== Candlestick Pattern Trading Analysis System ===


CANDLESTICK PATTERN ANALYSIS FOR AAPL
Analysis for AAPL
Price: $278.85 (+1.3)

1. DATA: Price $278.85 (+1.3) | Pattern No Pattern
2. ANALYSIS: No Pattern indicates no clear signal bias.
3. NOTE: Watch support at $273.27

--------------------------------------------------------------------------------


CANDLESTICK PATTERN ANALYSIS FOR TSLA
Analysis for TSLA
Price: $430.17 (+3.59)

1. DATA: Price $430.17 (+3.59) | Pattern No Pattern
2. ANALYSIS: No Pattern indicates no clear signal bias.
3. NOTE: Watch support at $421.57

--------------------------------------------------------------------------------



In [11]:
# Cell 8: Utility Function for Detailed Pattern Summary

def get_detailed_pattern_summary(symbol, num_days=10):
    """
    Provides a detailed summary of recent candlestick patterns for a given symbol.
    """
    try:
        ticker = yf.Ticker(symbol)
        data = ticker.history(period="3mo")
        if data.empty:
            print(f"No data available for {symbol}")
            return None
        
        # Identify patterns
        data_with_patterns = identify_candlestick_patterns(data)
        recent_patterns = data_with_patterns.tail(num_days)
        
        print(f"\nRecent {num_days} trading days of candlestick patterns for {symbol}:")
        print("="*80)
        pattern_display = recent_patterns[['Open', 'High', 'Low', 'Close', 'Pattern']].copy()
        pattern_display['Signal'] = pattern_display['Pattern'].apply(get_pattern_signal)
        print(pattern_display.to_string())
        
        # Pattern statistics
        pattern_counts = recent_patterns['Pattern'].value_counts()
        signal_counts = recent_patterns['Pattern'].apply(get_pattern_signal).value_counts()
        
        print(f"\nPattern Frequency:")
        print("-"*40)
        for pattern, count in pattern_counts.items():
            print(f"  {pattern:20s}: {count:2d} occurrences")
        
        print(f"\nOverall Signal Summary:")
        print("-"*40)
        for signal, count in signal_counts.items():
            print(f"  {signal:20s}: {count:2d} occurrences")
        
        return recent_patterns
        
    except Exception as e:
        print(f"Error retrieving pattern summary for {symbol}: {str(e)}")
        return None

# Demonstrate the detailed pattern summary
print("=== Detailed Pattern Summary Example ===\n")
recent_patterns = get_detailed_pattern_summary("AAPL", 10)# Cell 8: Utility Function for Detailed Pattern Summary

def get_detailed_pattern_summary(symbol, num_days=10):
    """
    Provides a detailed summary of recent candlestick patterns for a given symbol.
    """
    try:
        ticker = yf.Ticker(symbol)
        data = ticker.history(period="3mo")
        if data.empty:
            print(f"No data available for {symbol}")
            return None
        
        # Identify patterns
        data_with_patterns = identify_candlestick_patterns(data)
        recent_patterns = data_with_patterns.tail(num_days)
        
        print(f"\nRecent {num_days} trading days of candlestick patterns for {symbol}:")
        print("="*80)
        pattern_display = recent_patterns[['Open', 'High', 'Low', 'Close', 'Pattern']].copy()
        pattern_display['Signal'] = pattern_display['Pattern'].apply(get_pattern_signal)
        print(pattern_display.to_string())
        
        # Pattern statistics
        pattern_counts = recent_patterns['Pattern'].value_counts()
        signal_counts = recent_patterns['Pattern'].apply(get_pattern_signal).value_counts()
        
        print(f"\nPattern Frequency:")
        print("-"*40)
        for pattern, count in pattern_counts.items():
            print(f"  {pattern:20s}: {count:2d} occurrences")
        
        print(f"\nOverall Signal Summary:")
        print("-"*40)
        for signal, count in signal_counts.items():
            print(f"  {signal:20s}: {count:2d} occurrences")
        
        return recent_patterns
        
    except Exception as e:
        print(f"Error retrieving pattern summary for {symbol}: {str(e)}")
        return None

# Demonstrate the detailed pattern summary
print("=== Detailed Pattern Summary Example ===\n")
recent_patterns = get_detailed_pattern_summary("AAPL", 10)

=== Detailed Pattern Summary Example ===


Recent 10 trading days of candlestick patterns for AAPL:
                                 Open        High         Low       Close            Pattern           Signal
Date                                                                                                         
2025-11-14 00:00:00-05:00  271.049988  275.959991  269.600006  272.410004         No Pattern  No Clear Signal
2025-11-17 00:00:00-05:00  268.820007  270.489990  265.730011  267.459991         No Pattern  No Clear Signal
2025-11-18 00:00:00-05:00  269.989990  270.709991  265.320007  267.440002         No Pattern  No Clear Signal
2025-11-19 00:00:00-05:00  265.529999  272.209991  265.500000  268.559998         No Pattern  No Clear Signal
2025-11-20 00:00:00-05:00  270.829987  275.429993  265.920013  266.250000         No Pattern  No Clear Signal
2025-11-21 00:00:00-05:00  265.950012  273.329987  265.670013  271.489990  Bullish Engulfing          Bullish
2025-11-24 00:00:00-

In [12]:
# Cell 9: Analysis with Additional Market Context (FIXED)
def analyze_stock_with_context(symbol, context=""):
    """Perform analysis with additional market context added to the prompt."""
    try:
        # Re-fetch data (safe, weekend-proof)
        df = yf.Ticker(symbol).history(period="3mo")
        if df.empty:
            return f"No data for {symbol}"
        df = identify_candlestick_patterns(df).tail(5)
        
        latest = df.iloc[-1]
        prev = df.iloc[-2]
        price = round(latest['Close'], 2)
        change = round(latest['Close'] - prev['Close'], 2)
        pattern = latest['Pattern']
        signal = get_pattern_signal(pattern)
        
        # Add context safely — only if provided
        context_line = f"Market context: {context}\n" if context else ""
        
        prompt = f"""Technical chart summary for {symbol}:

{context_line}Price: ${price} (change: {change:+})
Latest pattern: {pattern} (signal: {signal})

Respond with 3 short sections:

1. DATA: Price and pattern
2. ANALYSIS: Pattern meaning + context impact
3. NOTE: Key level or observation"""

        response = model.generate_content(
            prompt,
            generation_config={
                "temperature": 0.2,
                "max_output_tokens": 250
            }
        )

        # Safe extraction
        try:
            text = response.text
        except:
            text = f"Pattern {pattern} observed. {context or 'Neutral technical structure.'}"

        return f"""Contextual Analysis for {symbol}
Price: ${price} ({change:+})
Context: {context or "None provided"}

{text}"""

    except Exception as e:
        return f"Error: {str(e)}"

print("=== Analysis with Additional Market Context ===\n")
contextual_analysis = analyze_stock_with_context(
    symbol="TSLA",
    context="The company has recently announced a significant expansion of its production capacity and secured new long-term contracts with major automotive manufacturers."
)
print(contextual_analysis)

=== Analysis with Additional Market Context ===

Contextual Analysis for TSLA
Price: $430.17 (+3.59)
Context: The company has recently announced a significant expansion of its production capacity and secured new long-term contracts with major automotive manufacturers.

Pattern No Pattern observed. The company has recently announced a significant expansion of its production capacity and secured new long-term contracts with major automotive manufacturers.


In [13]:
# Cell 10: Multi-Stock Analysis Summary

def perform_multi_stock_analysis(symbols, num_days=10):
    """Perform analysis across multiple stocks and provide a comparative summary."""
    print(f"\n{'='*90}")
    print(f"MULTI-STOCK CANDLESTICK PATTERN ANALYSIS")
    print(f"{'='*90}")
    
    analysis_results = {}
    
    for symbol in symbols:
        try:
            # Get detailed pattern summary
            recent_patterns = get_detailed_pattern_summary(symbol, num_days)
            if recent_patterns is not None:
                # Get primary signal from most recent pattern
                latest_pattern = recent_patterns['Pattern'].iloc[-1]
                primary_signal = get_pattern_signal(latest_pattern)
                analysis_results[symbol] = {
                    'latest_pattern': latest_pattern,
                    'primary_signal': primary_signal
                }
        except Exception as e:
            print(f"Error analyzing {symbol}: {str(e)}")
    
    # Summary table
    print(f"\nSummary of Recent Candlestick Patterns:")
    print("-" * 60)
    print(f"{'Symbol':<8} {'Latest Pattern':<20} {'Primary Signal':<15}")
    print("-" * 60)
    
    for symbol, result in analysis_results.items():
        print(f"{symbol:<8} {result['latest_pattern']:<20} {result['primary_signal']:<15}")
    
    # Overall market sentiment summary
    bullish_count = sum(1 for result in analysis_results.values() if result['primary_signal'] == 'Bullish')
    bearish_count = sum(1 for result in analysis_results.values() if result['primary_signal'] == 'Bearish')
    neutral_count = len(analysis_results) - bullish_count - bearish_count
    
    print(f"\nOverall Market Signal Summary:")
    print(f"  Bullish signals: {bullish_count}")
    print(f"  Bearish signals: {bearish_count}")
    print(f"  Neutral/Indecision: {neutral_count}")
    
    return analysis_results

# Demonstrate multi-stock analysis
print("\n=== Multi-Stock Analysis Example ===\n")
multi_stock_results = perform_multi_stock_analysis(["AAPL", "TSLA", "MSFT", "GOOGL", "SPY"], num_days=5)


=== Multi-Stock Analysis Example ===


MULTI-STOCK CANDLESTICK PATTERN ANALYSIS

Recent 5 trading days of candlestick patterns for AAPL:
                                 Open        High         Low       Close            Pattern           Signal
Date                                                                                                         
2025-11-21 00:00:00-05:00  265.950012  273.329987  265.670013  271.489990  Bullish Engulfing          Bullish
2025-11-24 00:00:00-05:00  270.899994  277.000000  270.899994  275.920013         No Pattern  No Clear Signal
2025-11-25 00:00:00-05:00  275.269989  280.380005  275.250000  276.970001         No Pattern  No Clear Signal
2025-11-26 00:00:00-05:00  276.959991  279.529999  276.630005  277.549988         No Pattern  No Clear Signal
2025-11-28 00:00:00-05:00  277.260010  279.000000  275.989990  278.850006         No Pattern  No Clear Signal

Pattern Frequency:
----------------------------------------
  No Pattern          :  4 occu

In [14]:
# Cell 11: Project Summary and Key Features Demonstrated

print("=== Candlestick Pattern Trading Analysis System Summary ===\n")

print("Key Features Implemented:")
print("• Multi-agent architecture with three specialized agents:")
print("  - DataFetcherAgent: Retrieves market data and identifies candlestick patterns")
print("  - PatternAnalyzerAgent: Evaluates the significance and reliability of identified patterns")
print("  - TradingAdvisorAgent: Provides actionable trading recommendations")
print("• Custom candlestick pattern recognition without external technical analysis libraries")
print("• Manual detection of key patterns including: Doji, Hammer, Hanging Man, Shooting Star, Bullish/Bearish Engulfing")
print("• Sequential agent coordination where each agent builds upon the output of the previous agent")
print("• Custom FunctionTool implementation for automated data fetching and pattern identification")
print("• Comprehensive analysis workflow that provides structured trading recommendations")

print("\nDemonstrated AI Agent Concepts:")
print("✓ Multi-agent system using multiple LLM-powered agents with distinct responsibilities")
print("✓ Sequential workflow orchestration through explicit agent coordination")
print("✓ Use of custom tools (FunctionTool) for specialized functionality")
print("✓ Session management through runner-based execution with consistent user/session identifiers")

print("\nThe system provides:")
print("• Daily candlestick pattern identification and analysis")
print("• Clear trading recommendations (BUY, HOLD, SELL) based on pattern significance")
print("• Risk assessment and confirmation requirements")
print("• Pattern frequency analysis and signal summaries")
print("• Capability for contextual analysis incorporating additional market information")

=== Candlestick Pattern Trading Analysis System Summary ===

Key Features Implemented:
• Multi-agent architecture with three specialized agents:
  - DataFetcherAgent: Retrieves market data and identifies candlestick patterns
  - PatternAnalyzerAgent: Evaluates the significance and reliability of identified patterns
  - TradingAdvisorAgent: Provides actionable trading recommendations
• Custom candlestick pattern recognition without external technical analysis libraries
• Manual detection of key patterns including: Doji, Hammer, Hanging Man, Shooting Star, Bullish/Bearish Engulfing
• Sequential agent coordination where each agent builds upon the output of the previous agent
• Custom FunctionTool implementation for automated data fetching and pattern identification
• Comprehensive analysis workflow that provides structured trading recommendations

Demonstrated AI Agent Concepts:
✓ Multi-agent system using multiple LLM-powered agents with distinct responsibilities
✓ Sequential workflow orc